In [4]:
DATA_DIR = "/opt/gpudata/skin-cancer/data"
OUTPUT_DIR = "/opt/gpudata/skin-cancer/outputs"

In [2]:
# load cross validation folds
import json
import os

with open(os.path.join(DATA_DIR, "folds.json"), "r") as f:
    folds = json.load(f)

In [141]:
# load slide embeddings
import pickle

with open(os.path.join(OUTPUT_DIR, "gigapath_slide_embeds_pool.pkl"), "rb") as f:
    embeds = pickle.load(f)

In [142]:
# map specimen id to a list of WSIs
specs = {}
for slide in embeds.keys():
    key = slide[:6]
    if specs.get(key) is not None:
        specs[key].append(slide)
    else:
        specs[key] = [slide]

In [143]:
slide_folds = [[] for _ in range(5)]
for i, fold in enumerate(folds):
    for spec in fold:
        slide_folds[i].extend(specs[spec])

In [144]:
# load data labels
import pandas as pd

labels = pd.read_csv(os.path.join(DATA_DIR, "labels/labels.csv"))
labels["specimenid"] = labels["specimenid"].astype("string")

In [145]:
# create dataframe with slide data
import numpy as np

slide_ids = list(embeds.keys())
specimen_ids = [slide_id[:6] for slide_id in slide_ids]
embeddings = [embeds[slide_id] for slide_id in slide_ids]
df = pd.DataFrame({"slide_id": slide_ids, "specimen_id": specimen_ids, "embedding": embeddings})
df["slide_id"] = df["slide_id"].astype("string")
df["specimen_id"] = df["specimen_id"].astype("string")
df["embedding"] = df["embedding"].apply(lambda x: np.array(x.squeeze(0)))

In [146]:
# merge slide data dataframe with labels dataframe
df = df.merge(
    labels[
        ["specimenid", "bowens1", "scc1", "bcc1"]
    ], how="right", left_on="specimen_id", right_on="specimenid",
)
df = df.set_index("slide_id")

In [148]:
from enum import IntEnum

class Label(IntEnum):
    na = 0
    bowens = 1
    bcc = 2
    scc = 3


def get_labels(x):
    if x["bowens1"].item() == 1:
        return Label.bowens.value
    elif x["bcc1"].item() == 1:
        return Label.bcc.value
    elif x["scc1"].item() == 1:
        return Label.scc.value
    else:
        return Label.na.value

In [149]:
X = df["embedding"]
y = df[["bowens1", "scc1", "bcc1"]]
y = y.apply(get_labels, axis=1)

In [159]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# clf = LogisticRegression(max_iter=500)
clf = XGBClassifier(objective="multi:softmax", num_class=4)

val_set = np.arange(len(slide_folds) - 1, -1, -1)
for i in range(len(slide_folds)):
    val_fold = slide_folds[val_set[i]]

    X_val = X.loc[val_fold]
    X_train = X[X.index.difference(X_val.index)]
    X_val = X_val.to_list()
    X_train = X_train.to_list()

    y_val = y.loc[val_fold]
    y_train = y[y.index.difference(y_val.index)]

    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)
    probs = clf.predict_proba(X_val)
    score = accuracy_score(y_val, preds)
    print(score)
    print(classification_report(y_val, preds))

0.5023696682464455
              precision    recall  f1-score   support

           0       0.58      0.61      0.59       143
           1       0.43      0.39      0.41       141
           2       0.47      0.54      0.51       101
           3       0.56      0.41      0.47        37

    accuracy                           0.50       422
   macro avg       0.51      0.49      0.49       422
weighted avg       0.50      0.50      0.50       422

0.5666003976143141
              precision    recall  f1-score   support

           0       0.56      0.71      0.62       148
           1       0.45      0.46      0.46       155
           2       0.63      0.50      0.56       142
           3       0.86      0.66      0.75        58

    accuracy                           0.57       503
   macro avg       0.62      0.58      0.60       503
weighted avg       0.58      0.57      0.57       503

0.4880952380952381
              precision    recall  f1-score   support

           0      